In [1]:
!pip install fastai

In [0]:
from fastai.text import *

In [2]:
!git clone https://github.com/Tariq60/LIAR-PLUS.git

Cloning into 'LIAR-PLUS'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 77 (delta 2), reused 0 (delta 0), pack-reused 68
Unpacking objects: 100% (77/77), done.


In [3]:
!pwd

/content


In [4]:
path = Path('/content/LIAR-PLUS/dataset')
path

PosixPath('/content/LIAR-PLUS/dataset')

###Binary Classification using transfer learning on LSTM network using only statement

In [152]:
df = pd.read_csv(path/'train2.tsv' , sep = '\t' , header= None)
df_val = pd.read_csv(path/'val2.tsv', sep = '\t' , header = None)
df = df.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_val = df_val.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df.loc[(df[2] == 'true') , 2] = 5
df.loc[(df[2] == 'mostly-true') , 2] = 4
df.loc[(df[2] == 'half-true') , 2] = 3
df.loc[(df[2] == 'barely-true') , 2] = 2
df.loc[(df[2] == 'false') , 2] = 1
df.loc[(df[2] == 'pants-fire') , 2] = 0
df.columns = ['label','text']

df['label'] = (df['label'] > 2).astype(int)

df_val.loc[(df_val[2] == 'true') , 2] = 5
df_val.loc[(df_val[2] == 'mostly-true') , 2] = 4
df_val.loc[(df_val[2] == 'half-true') , 2] = 3
df_val.loc[(df_val[2] == 'barely-true') , 2] = 2
df_val.loc[(df_val[2] == 'false') , 2] = 1
df_val.loc[(df_val[2] == 'pants-fire') , 2] = 0
df_val.columns = ['label','text']

df_val['label'] = (df_val['label'] > 2).astype(int)

df.head()
# df.label.unique()


,label,text
0,0,Says the Annies List political group supports ...
1,1,When did the decline of coal start? It started...
2,1,"Hillary Clinton agrees with John McCain ""by vo..."
3,0,Health care reform legislation is likely to ma...
4,1,The economic turnaround started at the end of ...


In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(path,train_df = df ,valid_df = df_val)
# Classifier model data
data_clas = TextClasDataBunch.from_df(path ,train_df = df ,valid_df = df_val ,vocab=data_lm.train_ds.vocab, bs=8)



In [154]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.7)
learn.fit_one_cycle(2, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.283076,3.490433,0.362919,00:08
1,3.691448,3.405994,0.373549,00:08


In [155]:
learn.unfreeze()
learn.fit_one_cycle(2, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.322602,3.287825,0.386049,00:11
1,3.182704,3.259063,0.388588,00:12


In [0]:
learn.save_encoder('enc')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.7)
learn.load_encoder('enc')

In [158]:
data_clas.show_batch()

text,target
"xxbos xxmaj hospitals , doctors , xxunk , surgeries and so xxunk are more extensively used and far more expensive in this country than they are in many other countries . ' ' \t health - care \t mitt - romney \t xxmaj former governor \t xxmaj massachusetts \t republican \t 34 \t 32 \t 58 \t 33 \t 19 \t a xxmaj fox xxmaj news xxmaj sunday interview \n",1
xxbos xxmaj the vast majority of the money i got was from small donors all across the country . ' ' \t campaign - finance \t barack - obama \t xxmaj president \t xxmaj illinois \t democrat \t 70 \t 71 \t 160 \t 163 \t 9 \t an interview with xxup cnbc 's xxmaj john xxmaj xxunk \n xxunk \t true \t xxmaj the xxmaj democrat - controlled,0
"xxbos xxmaj after xxmaj hurricane xxmaj katrina , xxmaj dallas and xxmaj san xxmaj antonio took in 5,000 to 10,000 refugees each , but ( xxmaj bill ) xxmaj white brought in 250,000 and put them up in his buddies empty apartment buildings and let the federal government pay their xxunk for over two years . xxmaj they did not return to xxmaj new xxmaj orleans , most of them",0
"xxbos xxmaj chicago xxmaj public xxmaj schools lead the country in high school graduation of an urban school system , match the xxmaj united xxmaj states of xxmaj america in kids graduating high school going on to a four - year college , and are producing eighth - grade leaders in the country in math gains , fourth - grade leaders in reading gains .",1
xxbos xxmaj washington can not hide from the fact that xxmaj congress hiked the national debt ceiling to $ xxunk trillion . xxmaj they can not hide from the fact . . . that the budget that they re going forward with has more than a $ 5 trillion in debt in it over the next five years .,1


In [159]:
learn.fit_one_cycle(2,5e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.670780,0.660690,0.612150,00:20
1,0.668234,0.664342,0.588006,00:21


In [160]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-3,5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.664280,0.667209,0.580997,00:23


In [161]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-4, 5e-4))

epoch,train_loss,valid_loss,accuracy,time
0,0.667177,0.660405,0.607477,00:48
1,0.637833,0.653565,0.623832,00:48


###6-Way Classification using Tranfer Learning using only statement

In [92]:
df = pd.read_csv(path/'train2.tsv' , sep = '\t' , header= None)
df_val = pd.read_csv(path/'val2.tsv', sep = '\t' , header = None)
df_test = pd.read_csv(path/'test2.tsv', sep='\t' , header=None)
df = df.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_val = df_val.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df_test = df_test.drop([0,1,4,5,6,7,8,9,10,11,12,13,14,15],axis=1)

df.loc[(df[2] == 'true') , 2] = 5
df.loc[(df[2] == 'mostly-true') , 2] = 4
df.loc[(df[2] == 'half-true') , 2] = 3
df.loc[(df[2] == 'barely-true') , 2] = 2
df.loc[(df[2] == 'false') , 2] = 1
df.loc[(df[2] == 'pants-fire') , 2] = 0
df.columns = ['label','text']

# df['label'] = (df['label'] > 2).astype(int)

df_val.loc[(df_val[2] == 'true') , 2] = 5
df_val.loc[(df_val[2] == 'mostly-true') , 2] = 4
df_val.loc[(df_val[2] == 'half-true') , 2] = 3
df_val.loc[(df_val[2] == 'barely-true') , 2] = 2
df_val.loc[(df_val[2] == 'false') , 2] = 1
df_val.loc[(df_val[2] == 'pants-fire') , 2] = 0
df_val.columns = ['label','text']

# df_val['label'] = (df_val['label'] > 2).astype(int)


df_test.loc[(df_test[2] == 'true') , 2] = 5
df_test.loc[(df_test[2] == 'mostly-true') , 2] = 4
df_test.loc[(df_test[2] == 'half-true') , 2] = 3
df_test.loc[(df_test[2] == 'barely-true') , 2] = 2
df_test.loc[(df_test[2] == 'false') , 2] = 1
df_test.loc[(df_test[2] == 'pants-fire') , 2] = 0
df_test.columns = ['label','text']

# df = df.dropna()
# df_val = df_val.dropna()
df.head()
# df.label.unique()
# df.count()

,label,text
0,1,Says the Annies List political group supports ...
1,3,When did the decline of coal start? It started...
2,4,"Hillary Clinton agrees with John McCain ""by vo..."
3,1,Health care reform legislation is likely to ma...
4,3,The economic turnaround started at the end of ...


In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(path,train_df = df ,valid_df = df_val , test_df = df_test)
# Classifier model data
data_clas = TextClasDataBunch.from_df(path ,train_df = df ,valid_df = df_val , test_df = df_test , vocab=data_lm.train_ds.vocab, bs=8)


In [94]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.316461,3.511127,0.349358,00:08
1,3.651322,3.359189,0.377372,00:08
2,3.305683,3.335279,0.379436,00:08


In [95]:
learn.unfreeze()
learn.fit_one_cycle(2, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.056601,3.263472,0.387528,00:12
1,2.884382,3.247146,0.390039,00:12


In [0]:
learn.save_encoder('enc1')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn.load_encoder('enc1')

In [98]:
data_clas.show_batch()

text,target
"xxbos xxmaj hospitals , doctors , xxunk , surgeries and so xxunk are more extensively used and far more expensive in this country than they are in many other countries . ' ' \t health - care \t mitt - romney \t xxmaj former governor \t xxmaj massachusetts \t republican \t 34 \t 32 \t 58 \t 33 \t 19 \t a xxmaj fox xxmaj news xxmaj sunday interview \n",4
"xxbos xxmaj people want the minimum wage , they want marriage equality , they want womens health care . xxmaj there s issues that actually that really do matter , social issues that matter to the people of this state . xxmaj so i know xxmaj barbara would support those issues , i know the governor has nt . xxmaj you ve seen the actions he s taken on them",4
"xxbos xxmaj if you give guns to gangs , you can get out of jail free , if ( xxmaj californias xxmaj proposition 57 ) passes . xxmaj if you do a drive - by shooting , you can get out of jail free , if xxmaj proposition 57 passes . xxmaj if you discharge guns on a school xxunk , you can get out of jail free .",1
"xxbos xxmaj in the mid 1990s , in our state , ( xxmaj florida was ) xxmaj no . 1 in violent crime in xxmaj america . ... xxmaj the only xxunk i had was for the citizens of xxmaj florida , who were xxunk to a violent crime every three minutes and 45 seconds .",5
"xxbos xxmaj after taking hundreds of thousands of dollars from xxmaj goldman xxmaj sachs and other xxmaj wall xxmaj street banks , ( xxmaj debbie xxmaj wasserman xxmaj schultz ) has voted to prevent the xxmaj consumer xxmaj financial xxmaj protection xxmaj bureau from regulating payday loans and addressing racial discrimination in car loans .",4


In [99]:
learn.fit_one_cycle(2,1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.730492,1.742059,0.238318,00:20
1,1.727738,1.774445,0.247664,00:20


In [100]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.717391,1.778294,0.261682,00:24


In [101]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.678621,1.816608,0.267134,00:48


In [102]:
pred1 = learn.get_preds(ds_type = DatasetType.Test,ordered = True)
pred1

[tensor([[0.0499, 0.2285, 0.1457, 0.2355, 0.1976, 0.1428],
         [0.0300, 0.1419, 0.1319, 0.2305, 0.2575, 0.2081],
         [0.0924, 0.1968, 0.1680, 0.1844, 0.1768, 0.1817],
         ...,
         [0.0510, 0.1742, 0.1929, 0.2594, 0.1769, 0.1457],
         [0.0637, 0.1989, 0.1887, 0.2217, 0.1783, 0.1487],
         [0.0736, 0.2165, 0.1967, 0.2199, 0.1512, 0.1421]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

###6-Way Classification using both the statement and the justification

In [104]:
df = pd.read_csv(path/'train2.tsv' , sep = '\t' , header= None)
df_val = pd.read_csv(path/'val2.tsv', sep = '\t' , header = None)
df_test = pd.read_csv(path/'test2.tsv', sep='\t' , header=None)
df = df.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df_val = df_val.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df_test = df_test.drop([0,1,4,5,6,7,8,9,10,11,12,13,14],axis=1)

df.loc[(df[2] == 'true') , 2] = 5
df.loc[(df[2] == 'mostly-true') , 2] = 4
df.loc[(df[2] == 'half-true') , 2] = 3
df.loc[(df[2] == 'barely-true') , 2] = 2
df.loc[(df[2] == 'false') , 2] = 1
df.loc[(df[2] == 'pants-fire') , 2] = 0
df.columns = ['label','text','just']

# df['label'] = (df['label'] > 2).astype(int)

df_val.loc[(df_val[2] == 'true') , 2] = 5
df_val.loc[(df_val[2] == 'mostly-true') , 2] = 4
df_val.loc[(df_val[2] == 'half-true') , 2] = 3
df_val.loc[(df_val[2] == 'barely-true') , 2] = 2
df_val.loc[(df_val[2] == 'false') , 2] = 1
df_val.loc[(df_val[2] == 'pants-fire') , 2] = 0
df_val.columns = ['label','text','just']

# df_val['label'] = (df_val['label'] > 2).astype(int)


df_test.loc[(df_test[2] == 'true') , 2] = 5
df_test.loc[(df_test[2] == 'mostly-true') , 2] = 4
df_test.loc[(df_test[2] == 'half-true') , 2] = 3
df_test.loc[(df_test[2] == 'barely-true') , 2] = 2
df_test.loc[(df_test[2] == 'false') , 2] = 1
df_test.loc[(df_test[2] == 'pants-fire') , 2] = 0
df_test.columns = ['label','text','just']

# df = df.dropna()
# df_val = df_val.dropna()
# df_test = df_test.dropna()
# df.head()
# df.label.unique()
# df_test.count()

df['just'] = df['just'].combine_first(df['text'])
df_val['just'] = df_val['just'].combine_first(df_val['text'])
df_test['just'] = df_test['just'].combine_first(df_test['text'])

df = df.drop(['text'],axis=1)
df_val = df_val.drop(['text'],axis=1)
df_test = df_test.drop(['text'],axis=1)
df.count()

label    10240
just     10240
dtype: int64

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(path,train_df = df ,valid_df = df_val , test_df = df_test)
# Classifier model data
data_clas = TextClasDataBunch.from_df(path ,train_df = df ,valid_df = df_val , test_df = df_test , vocab=data_lm.train_ds.vocab, bs=8)


In [106]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.7)
learn.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.108919,3.764016,0.309317,00:39
1,3.789429,3.660683,0.319610,00:40
2,3.503769,3.632346,0.323223,00:41


In [107]:
learn.unfreeze()
learn.fit_one_cycle(2, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.381448,3.565887,0.333061,00:54
1,3.210445,3.538427,0.337260,00:55


In [0]:
learn.save_encoder('enc1')

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.7)
learn.load_encoder('enc1')

In [110]:
data_clas.show_batch()

text,target
"xxbos xxmaj trump said , "" i do n't know anything about xxmaj david xxmaj duke . "" xxmaj trump seems to have forgotten that he knew enough about xxmaj duke to denounce him twice this election cycle . xxmaj in 2000 , he knew enough to criticize xxmaj xxunk racism . xxmaj and in 1991 , he knew enough to say he hated what votes for xxmaj duke represented",4
"xxbos xxmaj watson said , "" xxmaj more lives are lost from violence against women , sex - selective abortion , female infanticide , suicide , egregious maternal mortality , and other sex - linked causes than were lost during all of the wars and civil strife of 20th century . "" xxmaj that s 10 million more than her estimate of lives lost in all of last xxunk strife",4
"xxbos xxmaj the xxmaj daily xxmaj caller said that the xxmaj clinton xxmaj foundation and the xxmaj clinton xxmaj health xxmaj access xxmaj initiative distributed "" watered - down xxup hiv / xxup aids drugs to patients in sub - xxmaj saharan xxmaj africa . "" xxmaj at the most literal level , the statement is a misreading of a report from a conservative xxmaj republican who said the foundation",1
"xxbos xxmaj first , xxmaj dorworth oversimplified things when he said xxmaj florida reviews local land use decisions . xxmaj the state review is of changes to comprehensive plans . xxmaj second , xxmaj dorworth said only that xxunk be right that xxmaj florida is the only state in the country with a state review procedure . xxmaj our analysis shows that xxmaj florida has a unique form of state",2
"xxbos xxmaj republican xxmaj senate candidate xxmaj barry xxmaj hinckley said , "" 30 percent of [ xxmaj sheldon xxmaj whitehouse 's campaign ] money is coming from special interests . "" xxmaj if xxmaj hinckley was referring exclusively to political action committees , we would rate his statement xxmaj true . xxmaj but "" special - interest group "" is a broader -- if ill - defined -- category",2


In [111]:
learn.fit_one_cycle(2,1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.747650,1.797260,0.191589,01:00
1,1.747124,1.778223,0.199377,01:00


In [112]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.737710,1.762939,0.218069,01:08


In [113]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.764555,1.809247,0.211838,02:19


In [114]:
pred2 = learn.get_preds(ds_type = DatasetType.Test,ordered = True)
pred2

[tensor([[0.0865, 0.1498, 0.1850, 0.2059, 0.2301, 0.1427],
         [0.0793, 0.1756, 0.1762, 0.2092, 0.2137, 0.1460],
         [0.0755, 0.2010, 0.1660, 0.1996, 0.1950, 0.1629],
         ...,
         [0.0912, 0.1649, 0.1694, 0.2002, 0.2030, 0.1715],
         [0.0706, 0.1900, 0.1591, 0.2099, 0.2041, 0.1663],
         [0.0764, 0.1909, 0.1760, 0.1965, 0.1991, 0.1610]]),
 tensor([0, 0, 0,  ..., 0, 0, 0])]

In [124]:
pred2[1].size()

torch.Size([1267])

In [125]:
pred1[1].size()

torch.Size([1267])

In [0]:
pred = pred1[0] + pred2[0]

In [0]:
_,argmax = pred.max(-1)

In [0]:
argmax = argmax.numpy()

In [0]:
labels_test = df_test['label'].tolist()
labels_test = np.array(labels_test)

In [131]:
cur = 0
for i in range(len(argmax)):
    if argmax[i] == labels_test[i]:
        cur += 1

print("Accuracy: %.2f%%" % ((cur/float(len(labels_test)) * 100)))

Accuracy: 26.12%
